In [10]:
import numpy as np
import mdtraj as md
from openmm import app
import openmm as mm
from openmm import unit as u
import pdbfixer
from sys import stdout

In [11]:
# peptide = md.load('data/peptide_fixed.pdb')
# peptide_top = peptide.top
# selection_petide = peptide_top.select("symbol != H")
# new_petide = peptide.atom_slice(selection_petide)
# new_petide.save('data/petide_fixed_no_H.pdb')

In [12]:
STRIDE_water = 1000
STRIDE_no_water = 1000 
STRIDE_checkpoint = 10000
simulations_steps = 10000 
real_time_STRIDE = 1000

geometry = 'truncatedOctahedron'
geompadding = 2*1.2*u.nanometer

platform_name = 'CPU'#'OpenCL' 

temperature = 300 * u.kelvin#paper
friction_coeff = 1.0/u.picoseconds
time_step = 2.0*u.femtoseconds 
pressure = 1 * u.bar #paper

In [13]:
# Loading the file into the notebook.
pdb_file = 'data/peptide_fixed.pdb'#pdb databank
fixer = pdbfixer.PDBFixer(pdb_file)

In [14]:
#https://github.com/openmm/openmm-setup/blob/931bddaf9fa185e44c79af2c638c22eed3cbe7d4/openmmsetup/openmmsetup.py#L251-L256
maxSize = max(max((pos[i] for pos in fixer.positions))-min((pos[i] for pos in fixer.positions)) for i in range(3))

In [15]:
#https://github.com/openmm/openmm/issues/2745
if geometry == 'truncatedOctahedron':
    vectors = mm.Vec3(1,0,0), mm.Vec3(1/3,2*np.sqrt(2)/3,0), mm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
    v_box = [(maxSize+geompadding)*v for v in vectors]
elif geometry == 'rhombicDodecahedron':
    vectors = mm.Vec3(1,0,0), mm.Vec3(0,1,0), mm.Vec3(0.5,0.5,np.sqrt(2)/2)
    v_box = [(maxSize+geompadding)*v for v in vectors]

In [16]:
# Defining the forcfield and the model
forcefield = app.ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3pfb.xml')
model = app.Modeller(fixer.topology, fixer.positions)
model.addSolvent(forcefield,
             boxVectors=v_box,
             model='tip3p', 
             ionicStrength=0.1 * u.millimolar, 
             positiveIon='Na+',
             negativeIon='Cl-',
             neutralize=True )

app.PDBFile.writeFile(model.topology,model.positions, open('data/peptide_model_water.pdb', 'w'))

petide_in_water = md.load('data/peptide_model_water.pdb')
petide_in_water_top = petide_in_water.top
selection_petide = petide_in_water_top.select('protein')

system = forcefield.createSystem(model.topology, 
                                 nonbondedMethod=app.PME, 
                                 nonbondedCutoff=1.2*u.nanometers,#paper 
                                 constraints=app.HBonds, 
                                 rigidWater=True, 
                                 ewaldErrorTolerance=0.0005)

system.addForce(mm.MonteCarloBarostat(pressure, temperature))#paper

integrator = mm.LangevinIntegrator(temperature,
                                   friction_coeff,
                                   time_step)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName(platform_name)
simulation = app.Simulation(model.topology, system, integrator, platform)
simulation.context.setPositions(model.positions)
simulation.context.setPeriodicBoxVectors(*v_box)

simulation.minimizeEnergy()

simulation.reporters.append(mm.app.dcdreporter.DCDReporter('data/output_water.dcd',
                                                           STRIDE_water))
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('data/mm.log', 
                                                                       STRIDE_no_water, 
                                                                       step=True, 
                                                                       potentialEnergy=True, 
                                                                       temperature=True, 
                                                                       time=True))
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter(stdout,
                                                                       real_time_STRIDE, 
                                                                       step=True, 
                                                                       potentialEnergy=True, 
                                                                       temperature=True, 
                                                                       speed=True, 
                                                                       elapsedTime=True))
simulation.reporters.append(md.reporters.DCDReporter('data/output_no_water.dcd',
                                                     STRIDE_no_water, 
                                                     atomSubset=selection_petide))
simulation.reporters.append(mm.app.checkpointreporter.CheckpointReporter('data/checkpoint.chk',
                            STRIDE_checkpoint))


simulation.step(simulations_steps)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Elapsed Time (s)"
1000,-135634.05812756982,259.53515040283173,--,0.0
2000,-132369.42201485412,287.0160807897519,18,9.611845254898071
3000,-132119.18474477256,293.87831577004886,18,19.21307682991028
4000,-131568.79660797637,303.3500412794067,17.9,28.97549033164978
5000,-130724.76021763135,298.72508208889644,17.9,38.714417934417725
6000,-131385.88916563877,299.0678121136219,17.8,48.6133451461792
7000,-131422.02269812327,298.809505210189,17.8,58.24039101600647
8000,-130813.67621996604,295.95949405002773,17.8,68.00791120529175
9000,-131112.4951549601,298.71963805183145,17.8,77.68929290771484
10000,-131202.6923771218,297.6344319421661,17.8,87.17653179168701
